In [2]:
import numpy as np
from tensorflow.keras.layers import Dense,Input,Reshape,Flatten,Conv2D,MaxPool2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img,ImageDataGenerator
from tensorflow.keras.applications import ResNet50

In [4]:
gen = ImageDataGenerator(rotation_range=15,shear_range=0.2,zoom_range=.2)

generator = gen.flow_from_directory("../datasets/data/")

Found 808 images belonging to 4 classes.


In [5]:
generator.class_indices



{'Humans': 0, 'cats': 1, 'dogs': 2, 'horses': 3}

In [6]:
resnet_Model = ResNet50(include_top=False,weights="imagenet",input_shape=(256,256,3))

94773248/94765736 [==============================] - 8s 0us/step

In [8]:
generator.image_shape

(256, 256, 3)

In [9]:
for layer in resnet_Model.layers:
    layer.trainable = False

In [10]:
resnet_Model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 262, 262, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 128, 128, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 128, 128, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [11]:
in_layer = Input(shape=(256,256,3))
resnet_layer  = resnet_Model(in_layer)
flatten = Flatten()(resnet_layer)
d1 = Dense(units=400,activation="tanh")(flatten)
d2 = Dense(units=100,activation="tanh")(d1)
out_layer = Dense(units=4,activation="softmax")(d2)


In [12]:
from tensorflow.keras.models import Model
model = Model(in_layer,out_layer)


In [13]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
resnet50 (Model)             (None, 8, 8, 2048)        23587712  
_________________________________________________________________
flatten (Flatten)            (None, 131072)            0         
_________________________________________________________________
dense (Dense)                (None, 400)               52429200  
_________________________________________________________________
dense_1 (Dense)              (None, 100)               40100     
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 404       
Total params: 76,057,416
Trainable params: 52,469,704
Non-trainable params: 23,587,712
________________________________________

In [17]:
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])

In [20]:
model.fit_generator(generator, steps_per_epoch=20, epochs=5)

Epoch 1/5
20/20 [==============================] - 50s 2s/step - loss: 0.0645 - accuracy: 0.9805

In [21]:
! wget https://previews.123rf.com/images/piksel/piksel1301/piksel130100090/17604593-casual-young-man-standing.jpg


--2020-05-27 16:29:54--  https://previews.123rf.com/images/piksel/piksel1301/piksel130100090/17604593-casual-young-man-standing.jpg
Resolving previews.123rf.com (previews.123rf.com)... 104.90.157.222
Connecting to previews.123rf.com (previews.123rf.com)|104.90.157.222|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 76003 (74K) [image/jpeg]
Saving to: ‘17604593-casual-young-man-standing.jpg’

17604593-casual-you 100%[===================>]  74.22K   197KB/s    in 0.4s    

2020-05-27 16:29:56 (197 KB/s) - ‘17604593-casual-young-man-standing.jpg’ saved [76003/76003]



In [22]:
img = np.array(load_img("17604593-casual-young-man-standing.jpg", target_size=(256, 256)))




In [23]:
img.shape

(256, 256, 3)

In [25]:
np.argmax(model.predict(np.array([img])),axis=1)

array([0])

In [27]:
generator.class_indices

{'Humans': 0, 'cats': 1, 'dogs': 2, 'horses': 3}